In [14]:
import pandas as pd
import boto3
from boto3.dynamodb.conditions import Key
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class Process:
    def __init__(self, config_file):
        self.config_file = config_file

    def delete_duplicates(self, dup_file_name):
        # config = self.__get_config()
        TABLE_NAME = 'prod-210011-loan-domain-data-table'
        dynamodb = boto3.resource(
            'dynamodb', region_name="us-east-2", verify=False)
        table = dynamodb.Table(TABLE_NAME)
        for duplicateLoanList in self.__read_csv(dup_file_name):
            for duplicateRlId in duplicateLoanList:
                response = table.query(KeyConditionExpression=Key('RocketLogicLoanId').eq(duplicateRlId))
                # print(response)
                for item in response['Items']:
                    table.delete_item(Key = {
                        'RocketLogicLoanId': item['RocketLogicLoanId'],
                        'SortKey': item['SortKey']
                    })
                    print(f"Completed {item['RocketLogicLoanId']}")

    # def __get_config(self):
    #     config_file_handle = open(self.config_file)
    #     return json.load(config_file_handle)

    def __read_csv(self, file_name, start_at=-1, stop_at=-1, batch_size=-1):
        if start_at != -1 or stop_at != -1 or batch_size != -1:
            number_of_rows_to_get = stop_at - start_at
            start_row = start_at - 1
        else:
            number_of_rows_to_get = None
            batch_size = None
            start_row = None
        reader = pd.read_csv(file_name, chunksize=batch_size, nrows=number_of_rows_to_get, skiprows=start_row,
                             skipinitialspace=True, iterator=True)
        for df in reader:
            yield df.iloc[:, 0].tolist()


process = Process('backfill.json')
process.delete_duplicates(
    '/Users/BRadhakrishnan/Documents/repo/personal/LoanPurposeBackfill/new_duplicates_1.csv')


Completed 86b406de-41e6-4d16-9044-5d12ff552af4
Completed eccf2c49-ee37-4b22-9c31-1aa6f4f49b7b
Completed 7d76c853-67ae-435d-aee3-cd7ba459e002
Completed 7d76c853-67ae-435d-aee3-cd7ba459e002
Completed a220919d-8325-47c2-94aa-6aad1b37b5f0
Completed 23b5c2b8-3735-4ad3-86d9-17e3b9c69dbe
Completed 886d80ce-b04d-4808-8bfa-fb74866eb013
Completed 565801f5-3e9b-4410-a447-ba094e224be1
Completed 7a43f03d-7a4a-42a5-a4a7-191f94bdf6fe
Completed d197f084-a18d-4995-b4cb-a128e531b951
Completed cf66f5e5-3afc-4033-8525-fb8651e0517d
Completed 669ad33a-9821-4d28-b49a-3fab890becfd
Completed 669ad33a-9821-4d28-b49a-3fab890becfd
Completed 8b5d084c-9507-402a-bd84-00d8c199b1e9
Completed 6ade23e9-2606-4ee2-9e37-63e86abd379d
Completed 6f6cdd11-db97-4b44-b818-77f4d3b0fe71
Completed 181aab60-dd3d-4bb8-9529-7bd690ba209d
Completed dea7a600-0f78-4efb-8e7c-446a6b36ffee
Completed 3baa2551-d14a-4c19-8bb5-641ea7206388
Completed cffdf57d-9818-46a2-9a3e-dd105b2125db
Completed cffdf57d-9818-46a2-9a3e-dd105b2125db
Completed cff

In [4]:
import gzip;

class LoanDomainRecord:
  RocketLogicLoanId = ''
  LoanNumber = '',
  Sortkey = '',
  LastUpdatedBy = '',
  SourceId = '',
  LastUpdateDateTime = '',
  CreatedBy = '',
  UniversalLoanIdentifier = '',
  CreatedDateTime = '',
  LoanPurpose = '',
  RelationshipWithSeller = '',
  PurchaseGoals = [],
  EscrowWaivers = [],
  RefinanceGoals = [],
  CashOutReasons = [],
  Version = ''
  pass


def read_ld_dump(file_name):
  with gzip.open(file_name) as file:
    for line in file:
      print(bytes.decode(line))
      break


read_ld_dump(
    '/Users/BRadhakrishnan/Documents/repo/personal/LoanPurposeBackfill/loandomain/2os3rq6kfe6ojmw75afkltz5zy.json.gz')


{"Item":{"RocketLogicLoanId":{"S":"88bef609-19dc-459d-906c-a0d593b000b4"},"SortKey":{"S":"4"},"LoanNumber":{"S":"3504352400"},"SourceId":{"S":"rl-api-qApp Orchestrator-Prod-206156"},"LastUpdateDateTime":{"S":"2022-04-08T19:24:25.769Z"},"RefinanceGoals":{"L":[{"S":"CashOut"}]},"CashOutReasons":{"L":[]},"DesiredCashOutAmount":{"N":"10000"},"Version":{"N":"4"},"CreatedBy":{"S":"rl-api-qApp Orchestrator-Prod-206156"},"UniversalLoanIdentifier":{"S":"549300FGXN1K3HLB1R50350435240049"},"CreatedDateTime":{"S":"2022-04-08T19:24:25.769Z"},"LoanPurpose":{"S":"Refinance"},"aws:rep:deleting":{"BOOL":false},"aws:rep:updatetime":{"N":"1649445869.341001"},"aws:rep:updateregion":{"S":"us-east-2"}}}

